<a href="https://colab.research.google.com/github/knitemblazor/sentiment-analysis-moview-review-Pang-Lee/blob/master/movie_data_set_pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np
from gensim.models import Word2Vec
import tensorflow as tf
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


combining all text contents to a file

In [0]:
from os import listdir

def load_doc(filename):
  file = open(filename, 'r')
  text = file.read()
  file.close()
  return text

counter=0
directory = '/content/gdrive/My Drive/latest/txt_sentoken/neg'
for filename in listdir(directory):
  counter=counter+1
  if not filename.endswith(".txt"):
    continue
  if(counter>20):
    break
  path = directory + '/' + filename
  doc = load_doc(path)
  print('Loaded %s' % filename)

In [0]:
filename = '/content/gdrive/My Drive/latest/moviepsen.txt'
file = open(filename,"r" )
negative = str(file.read())
file.close() 

In [0]:
filename = '/content/gdrive/My Drive/latest/moviensen.txt'
file = open(filename,"r" )
positive = str(file.read())
file.close() 

paragraph to sentence tokenizer

In [0]:
from pprint import pprint
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer
 
trainer = PunktTrainer()
trainer.INCLUDE_ALL_COLLOCS = True
trainer.train(positive)
 
tokenizer = PunktSentenceTokenizer(trainer.get_params())
pos=tokenizer.tokenize(positive)

In [0]:
trainer = PunktTrainer()
trainer.INCLUDE_ALL_COLLOCS = True
trainer.train(negative)
 
tokenizer = PunktSentenceTokenizer(trainer.get_params())
neg=tokenizer.tokenize(negative)

In [0]:
print (len(pos))

35941


In [0]:
print(len(neg))

35419


In [0]:
print(p)

in a regrettable crime of passion he takes a baseball bat to the guy's head , and suddenly finds himself a fugitive for murder , and estranged from his wife and two kids .


Removing stopwords and punctuations

In [0]:
import string
a=[]
filename = '/content/gdrive/My Drive/latest/stopwords.txt'
file = open(filename,"r" )
stopwords = file.read().splitlines()
file.close() 
for c in string.punctuation:
    a.append(c)
with open('/content/gdrive/My Drive/latest/posmov1f.txt', 'w') as f:
    for item in pos:
      words=(set(item.split()).difference(a))
      wordsf=" ".join(str(x) for x in list(words))
      f.write("%s\n" % wordsf)
file.close()

In [0]:
import string
a=[]
filename = '/content/gdrive/My Drive/latest/stopwords.txt'
file = open(filename,"r" )
stopwords = file.read().splitlines()
file.close() 
for c in string.punctuation:
    a.append(c)
with open('/content/gdrive/My Drive/latest/negmovf.txt', 'w') as f:
    for item in neg:
      words=(set(item.split()).difference(a))
      words=set(words).difference(stopwords)
      wordsf=" ".join(str(x) for x in list(words))
      f.write("%s\n" % wordsf)
file.close()

accessing processed files

In [0]:
filename = '/content/gdrive/My Drive/latest/posmovf.txt'
file = open(filename,"r" )
sentences = (file.read().splitlines())
file.close()

filename = '/content/gdrive/My Drive/latest/negmovf.txt'
file = open(filename,"r" )
sent2 = (file.read().splitlines())
file.close()

label=[]
for i in range(len(sentences)):
  label.append(1)
  
for i in range(len(sent2)):
  label.append(0)
for i in range(len(sent2)):
  sentences.append(sent2[i])

In [0]:
print(len(sentences))

71360


In [0]:

print((sentences[1]))

head finds two his to from bat kids of a takes himself regrettable guy's baseball passion murder the for estranged wife suddenly and fugitive crime he in


In [0]:
sense=[]
for i in range(len(sentences)):
    sense.append(0)
for i in range(0,len(sentences)):
    sense[i]=sentences[i].split()

In [0]:
sentencesf=[]
labelf=[]
j=[]
for i in range(len(sentences)):
  j.append(len(sense[i]))


for i in range(len(j)):
  if(j[i]<=68):
    sentencesf.append(sense[i])
    labelf.append(label[i])

In [0]:
model = Word2Vec(sentencesf,size=68, min_count=1)#word vector dim=28
# summarize the loaded model
print(model)
# summarize vocabulary
words = list(model.wv.vocab)
print(len(words))

Word2Vec(vocab=50825, size=68, alpha=0.025)
50825


In [0]:
zerovector=[]
for i in range(68):
  zerovector.append(0)
# print(label)
def sentencevector(sent):
  vector=[]
  for word in sent:
       vector.append(model[word])
  if(len(sent)<68):
    for i in range(len(sent),68):
      vector.append(zerovector)
  return(vector)
Xtotal=[]
Ytotal=labelf
for i in range(len(sentencesf)):
  a=np.array(sentencevector(sentencesf[i]))
  Xtotal.append(a)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [0]:
len(Xtotal)  

71338

In [0]:
import random
indices=[]
for i in range(len(Xtotal)):
  indices.append(i)
print(len(indices))

train = random.sample(indices,50000)
print(train)
test=list(set(indices).difference(train))
print(test)



In [0]:
Xtrain=[]
Xtest=[]
Ytrain=[]
Ytest=[]
for i in train:
  Xtrain.append(Xtotal[i])
  Ytrain.append(labelf[i])
for i in test:
  Xtest.append(Xtotal[i])
  Ytest.append(labelf[i])
 
Xtrain=np.array(Xtrain)
Xtest=np.array(Xtest)
Y_train=np.array(Ytrain)
Y_test=np.array(Ytest)

In [0]:
X_train = Xtrain.reshape(Xtrain.shape[0], 68, 68 , 1).astype('float32')
X_test=Xtest.reshape(Xtest.shape[0], 68, 68 , 1).astype('float32')

In [0]:
print(len(X_train),len(Y_train))

50000 50000


In [0]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(300, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(100, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=10)
model.evaluate(X_test, Y_test)


Epoch 1/10
50000/50000 [==============================] - 38s 755us/step - loss: 0.2059 - acc: 0.9157
Epoch 2/10
50000/50000 [==============================] - 35s 691us/step - loss: 0.1341 - acc: 0.9426
Epoch 3/10
50000/50000 [==============================] - 34s 688us/step - loss: 0.1211 - acc: 0.9487
Epoch 4/10
50000/50000 [==============================] - 34s 684us/step - loss: 0.1130 - acc: 0.9510
Epoch 5/10
50000/50000 [==============================] - 35s 709us/step - loss: 0.1065 - acc: 0.9525
Epoch 6/10
50000/50000 [==============================] - 36s 729us/step - loss: 0.1026 - acc: 0.9540
Epoch 7/10
50000/50000 [==============================] - 36s 714us/step - loss: 0.0986 - acc: 0.9554
Epoch 8/10
50000/50000 [==============================] - 35s 708us/step - loss: 0.0940 - acc: 0.9570
Epoch 9/10
50000/50000 [==============================] - 35s 705us/step - loss: 0.0911 - acc: 0.9571
Epoch 10/10
21338/21338 [==============================] - 3s 150us/step


[0.12334351781940227, 0.9499015839949735]

In [0]:
y_pred= model.predict_classes(X_test)

In [0]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.98      0.95     10566
           1       0.98      0.92      0.95     10772

   micro avg       0.95      0.95      0.95     21338
   macro avg       0.95      0.95      0.95     21338
weighted avg       0.95      0.95      0.95     21338

